In [1]:
import os,time
os.environ['CUDA_VISIBLE_DEVICES']='0'
import copy
import logging
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date
import time
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, \
    default_data_collator, TrainingArguments, Trainer
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from scipy.stats import pearsonr, spearmanr
from transformers import AutoTokenizer
from transformers import BertConfig, BertTokenizer,BertTokenizerFast
from datasets import Dataset
from typing import List,Union
import tqdm
import re
import json

os.chdir('/home/stops/Work_space/NLP_work/Med_assit_chatglm')


from db_config_taiyi import DB ## load data from pgsql

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')
logger.info('Starting')


[2023-09-25 11:24:23,966] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-09-25 11:24:28,725 - INFO : Starting


In [2]:
model_path="clf_checkpoint/model_best/"


In [3]:
"""
labels = {0: "code", 1: "brainstorming", 2: "health", 3: "chat", 4: "translation", 5: "rewriting",
          6: "summarization", 7: "expansion", 8: "extraction", 9: "classification", 10: "science", 11: "math",
          12: "biography", 13: "business", 14: "society", 15: "economy", 16: "culture", 17: "other"}

"""


labels = {0: "代码", 1: "头脑风暴", 2: "医疗健康", 3: "闲聊", 4: "翻译", 5: "重写",
          6: "摘要总结", 7: "扩写", 8: "信息抽取", 9: "文本分类", 10: "科学", 11: "数学",
          12: "地理", 13: "商业", 14: "社会", 15: "经济", 16: "文化", 17: "其他"}

labels2zh = {"code": "代码", "brainstorming": "头脑风暴", "health": "医疗健康", "chat": "闲聊", "translation": "翻译", "rewriting": "重写",
          "summarization": "摘要总结", "expansion": "扩写", "extraction": "信息抽取", "classification": "文本分类", "science": "科学", "math": "数学",
          "biography": "传记", "business": "商业", "society": "社会", "economy": "经济", "culture": "文化", "other": "其他"}


In [4]:
from Query_clf_model import Query_clf_model,softmax


class Query_clf_Infer():
    def __init__(self,model_path,label2id_file='data/type_label2id_0206.json',batch_size=300):
        self.model_path=model_path
        self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer=BertTokenizerFast.from_pretrained(self.model_path)
        self.model= Query_clf_model.from_pretrained(self.model_path)
        self.model.eval()
        self.model.to(self.device)
        self.batch_size=batch_size
        self.label2id_file=label2id_file
        if self.label2id_file:
            with open(self.label2id_file,'r') as f :
                self.label2id=json.load(f)
            self.id2label=dict(zip(self.label2id.values(),self.label2id.keys()))
            self.num_classes=len(self.id2label)
        else:
            self.label2id=None
            self.id2label=None
            self.num_classes =2
        logger.info('load model and tokenizer : {}, num_classes: {}'.format(self.model_path,self.num_classes))

    def batch_infer(self,query:Union[str,List[str]]):
        if isinstance(query,str):
            query=[query]
        max_seq_len=max([len(query[i]) for i in range(len(query))])
        max_seq_len=min(max_seq_len,510)
        tokenized_examples=self.tokenizer(query,padding="max_length",max_length=max_seq_len+2,truncation=True)
        all_logits=np.empty(shape=[0,self.num_classes])
        with torch.no_grad():
            for idx in range(0,len(query),self.batch_size):
                bth_input_ids=torch.as_tensor(tokenized_examples['input_ids'][idx:idx+self.batch_size])
                bth_token_type_ids=torch.as_tensor(tokenized_examples['token_type_ids'][idx:idx+self.batch_size])
                bth_attent_mask=torch.as_tensor(tokenized_examples['attention_mask'][idx:idx+self.batch_size])
                logits_tensor= self.model(
                    input_ids=bth_input_ids.to(self.device),
                    token_type_ids=bth_token_type_ids.to(self.device),
                    attention_mask=bth_attent_mask.to(self.device))
                logits_np=logits_tensor.cpu().numpy()
                all_logits= np.append(all_logits,logits_np, axis = 0)
        batch_pred_proba = softmax(all_logits)
        batch_pred_class = batch_pred_proba.argmax(axis=1)
        if self.id2label:
            batch_pred_class=[self.id2label.get(item) for item in batch_pred_class]
        batch_pred_probability=batch_pred_proba.max(axis=1)
        batch_pred_probability=[round(item,4) for item in batch_pred_probability]
        return batch_pred_probability,batch_pred_class


In [5]:
label2id_file = "data/instruct_label2id.json"
QCI = Query_clf_Infer(model_path,label2id_file)

2023-09-25 11:24:31,963 - INFO : load model and tokenizer : clf_checkpoint/model_best/, num_classes: 18


In [6]:
s_time = time.time()
pred_question_bth = ['人工智能是什么',
                     '我的卡怎么没法用呀']
all_predictions = QCI.batch_infer(pred_question_bth)
print('all_predictions: ', all_predictions)
for i in range(len(all_predictions)):
    print('***' * 15)
    print('Group index : ', i)
    print('Question    : ', pred_question_bth[i])
    print('Proba       : {}, Label:  {}  '.format(all_predictions[0][i], all_predictions[1][i]))
print('cost_time: ', time.time() - s_time)


all_predictions:  ([0.3521, 0.7359], ['other', 'other'])
*********************************************
Group index :  0
Question    :  人工智能是什么
Proba       : 0.3521, Label:  other  
*********************************************
Group index :  1
Question    :  我的卡怎么没法用呀
Proba       : 0.7359, Label:  other  
cost_time:  0.49480772018432617


In [7]:
##################################
## Instruct data
##################################

data_file='/home/stops/Work_space/NLP_models/train_2M_CN/train_2M_CN.json'

belle_instruct_data=[]
with open(data_file,'r') as f :
    for sub_data in f.readlines()[:50000]:
        sub_data_dict=json.loads(sub_data)
        belle_instruct_data.append(sub_data_dict)
logger.info(f'belle_instruct_data nums: {len(belle_instruct_data)}')
logger.info(f'examples: {belle_instruct_data[:2]}')

2023-09-25 11:24:39,370 - INFO : belle_instruct_data nums: 50000
2023-09-25 11:24:39,371 - INFO : examples: [{'instruction': '对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  print(add(2,3))', 'input': '', 'output': '这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是说函数在 `return` 语句处结束，因此 `print` 语句永远不会被执行。\n要解决这个问题，只需在 `return` 语句之前移到 `print` 语句。修改后的代码如下所示：\n```\ndef add(a, b):\n    print(a + b)\n    return a + b\nadd(2, 3)\n```\n这样就可以在调用 `add` 函数时打印出结果，并在函数内部返回 `a + b` 值，使其在需要时可以接收和使用。同时，注意在调用 `add` 函数时，不需要再使用 `print` 方法了。'}, {'instruction': '为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。\n', 'input': '', 'output': '为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜太长；\n2. 名称应该与应用程序的主要功能或特点相关；\n3. 名称应该引起人们的兴趣和好奇心。\n举个例子，如果我们的应用程序是一款健康管理应用程序，那么可以考虑以下几个名称：\n1. HealthBoss\n2. FitMeUp\n3. HealthMate\n4. BodyBest\n5. HealthLab\n其中，HealthBoss强调应用程序可帮助用户成为健康的老板，FitMeUp强调应用程序可帮助用户达到健康生活的目标，HealthMate强调应用程序是用户健康的伙伴，BodyBest强调应用程序可通过健康管理帮助用户获得最佳身体状态，HealthLab强调应用程序可帮助用户进行健康诊断和实验。\n一句简短的介绍可包含以下几个要素：\n1. 应用程序的名称；\n2. 应用程序的

In [8]:
belle_instruct_df=pd.DataFrame.from_dict(belle_instruct_data)
belle_instruct_df=belle_instruct_df.applymap(lambda x:x.strip())
print(belle_instruct_df.shape)
print(belle_instruct_df.head())
print(belle_instruct_df.isnull().sum())

(50000, 3)
                                         instruction input  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...         
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。         
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...         
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...         
4                                  回答下面这个问题：什么是人工智能？         

                                              output  
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...  
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...  
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...  
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。  
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...  
instruction    0
input          0
output         0
dtype: int64


/tmp/ipykernel_1915/3953810507.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  belle_instruct_df=belle_instruct_df.applymap(lambda x:x.strip())


In [9]:
belle_instruct_df.loc[:,'instruct_len']=belle_instruct_df.apply(lambda x:len(x['instruction']),axis=1)
print(belle_instruct_df['instruct_len'].describe(percentiles=[0.01,0.1,0.2,0.25,0.5,0.75,0.8,0.9,0.9999]))

count     50000.000000
mean         49.287700
std          32.369842
min           6.000000
1%           13.000000
10%          21.000000
20%          26.000000
25%          29.000000
50%          42.000000
75%          60.000000
80%          65.000000
90%          84.000000
99.99%      576.000600
max        1004.000000
Name: instruct_len, dtype: float64


In [10]:
select_instruct_df=belle_instruct_df.loc[belle_instruct_df['instruct_len']<500,['instruction','output']].copy()
print(select_instruct_df.shape)
print(select_instruct_df.head())


(49993, 2)
                                         instruction  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...   
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。   
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...   
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...   
4                                  回答下面这个问题：什么是人工智能？   

                                              output  
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...  
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...  
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...  
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。  
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...  


In [11]:
####################
## 批次预测结果
####################

text_list=select_instruct_df["instruction"].tolist()
infer_class_result=[]
infer_proba_result=[]
batch_size=20
all_nums=len(text_list)
logger.info(f"run nums: {all_nums} , batch_size: {batch_size}")

for idx in range(0,all_nums,batch_size):
    if idx%100==0:
        logger.info("run step: {}, finished: {:.2%}".format(idx,idx/all_nums))
    sub_batch_texts=text_list[idx:(idx+batch_size)]
    sub_infer_texts=QCI.batch_infer(sub_batch_texts)
    for sub_proba,sub_class in zip(sub_infer_texts[0],sub_infer_texts[1]):
        infer_proba_result.append(sub_proba)
        infer_class_result.append(sub_class)


logger.info(f"infer_proba_result nums: {len(infer_proba_result)}")
logger.info(f"examples: {infer_proba_result[:2]}")
logger.info(f"examples: {infer_class_result[:2]}")


2023-09-25 11:24:39,767 - INFO : run nums: 49993 , batch_size: 20
2023-09-25 11:24:39,767 - INFO : run step: 0, finished: 0.00%
2023-09-25 11:24:39,810 - INFO : run step: 20, finished: 0.04%
2023-09-25 11:24:39,842 - INFO : run step: 40, finished: 0.08%
2023-09-25 11:24:39,923 - INFO : run step: 60, finished: 0.12%
2023-09-25 11:24:39,966 - INFO : run step: 80, finished: 0.16%
2023-09-25 11:24:40,014 - INFO : run step: 100, finished: 0.20%
2023-09-25 11:24:40,056 - INFO : run step: 120, finished: 0.24%
2023-09-25 11:24:40,093 - INFO : run step: 140, finished: 0.28%
2023-09-25 11:24:40,132 - INFO : run step: 160, finished: 0.32%
2023-09-25 11:24:40,156 - INFO : run step: 180, finished: 0.36%
2023-09-25 11:24:40,219 - INFO : run step: 200, finished: 0.40%
2023-09-25 11:24:40,275 - INFO : run step: 220, finished: 0.44%
2023-09-25 11:24:40,319 - INFO : run step: 240, finished: 0.48%
2023-09-25 11:24:40,346 - INFO : run step: 260, finished: 0.52%
2023-09-25 11:24:40,399 - INFO : run step: 2

In [12]:
select_instruct_df.loc[:,"proba"]=infer_proba_result
select_instruct_df.loc[:,"label"]=infer_class_result
select_instruct_df.loc[:,"label_zh"]=select_instruct_df.loc[:,"label"].map(labels2zh)

print(select_instruct_df.shape)
print(select_instruct_df.head())
print(select_instruct_df.isnull().sum())



(49993, 5)
                                         instruction  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...   
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。   
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...   
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...   
4                                  回答下面这个问题：什么是人工智能？   

                                              output   proba       label  \
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...  0.9386        code   
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...  0.5848   biography   
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...  0.9206        code   
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。  0.4707   rewriting   
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...  0.3840  extraction   

  label_zh  
0       代码  
1       传记  
2       代码  
3       重写  
4     信息抽取  
instruction    0
output         0
proba          0
label          0
label_zh       0
dtype: int64


In [13]:
save_mode=True
save_file="output_data/Instruct_prediction_clf_0925_5w_df.xlsx"
if save_mode:
    select_instruct_df.to_excel(save_file,index=False)
    logger.info(f"data nums: {select_instruct_df.shape}, save file: {save_file}")


2023-09-25 11:26:54,747 - INFO : data nums: (49993, 5), save file: output_data/Instruct_prediction_clf_0925_5w_df.xlsx
